In [1]:
from lib.jianji.code_csf import csf
from lib.model.alexnet import AlexNet
from lib.jianji.tool import *
import torch 
from torch.utils.data import Dataset
from torch import nn
from torchsummary import summary
from d2l import torch as d2l

In [2]:
traindSet = csf(csv_path='original_train_set.pkl', mode='train')
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=256, drop_last=True, shuffle=True)
testset = csf(csv_path='test_valid_edited(30000).pkl', mode='valid')
testloader = torch.utils.data.DataLoader(testset, batch_size=256, drop_last=True, shuffle=True)

In [3]:
lr = 0.5
num_epochs = 2
net = AlexNet()
device = d2l.try_gpu()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
loss = nn.BCELoss()
train_from_scratch(net, optimizer, loss, num_epochs, trainloader, device)
evalute_results(net, testloader, 0.1, device)

0.853243015680212

In [13]:
evalute_results(net, testloader, 0.48, device)

0.6013955662393162

In [11]:
torch.save(net.state_dict(), '11_1_alexnet.pth')

In [5]:
testset = csf(csv_path='test_valid_edited(30000).pkl', mode='valid')
testloader = torch.utils.data.DataLoader(testset, batch_size=256, drop_last=True, shuffle=True)

In [6]:
evalute_results(net, testloader, 0.1, device)

0.5885416666666666

In [15]:
pip install scikit-optimize

     ------------------------------------- 100.3/100.3 kB 36.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# 定义超参数空间
param_space = {
    'learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'epochs': Integer(1, 3)
}

# 定义评估函数（模拟您的深度学习模型性能评估）
def evalute_results(net, data_iter, threshold_value, device): ##最后输出一个值，模型在验证集上的平均正确率,默认模型在显卡上
    net.eval()
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            X = X.to(device)
            y = y.to(device)
            output = net(X)
            y_pred = torch.where(output>threshold_value,torch.ones_like(output),torch.zeros_like(output))
            cmp = d2l.astype(y_pred, y.dtype) == y
            acc_num = d2l.reduce_sum(d2l.astype(cmp, y.dtype))
            metric.add(acc_num, y.numel())
    return metric[0] / metric[1]

# 创建贝叶斯搜索对象
bayes_search = BayesSearchCV(
    evaluate_results,
    param_space,
    n_iter=20,  # 指定迭代次数
    random_state=0  # 随机数生成器的种子
)

# 运行贝叶斯搜索来找到最佳超参数组合
bayes_search.fit(X, y)  # X和y是您的训练数据和标签

# 打印最佳超参数组合
best_params = bayes_search.best_params_
print("Best hyperparameters:", best_params)